In [18]:
!pip install biopython
from Bio import SeqIO
from Bio.Seq import Seq

In [4]:
from google.colab import files
upload = files.upload()
# lambda_virus.fa

Saving lambda_virus.fa to lambda_virus.fa


A function that parses a DNA reference genome from a file in the FASTA format.



In [22]:
# Load the lambda genome (replace 'lambda.fasta' with your file)
lambda_genome = SeqIO.read("lambda_virus.fa", "fasta").seq

In [24]:
print(lambda_genome)
print(len(lambda_genome))

...
48502


In [30]:
# Define the query sequences
query = "TTAA"
reverse_complement = str(Seq(query).reverse_complement())  # "AGGT" => "ACCT"

In [31]:
# Count occurrences
count_query = lambda_genome.count(query)
count_rc = lambda_genome.count(reverse_complement)

In [32]:
total_occurrences = count_query + count_rc
print(f"count 1 '{count_query}' and its inverse '{count_rc}' "  )

print(f"Total occurrences of '{query}' or its reverse complement: {total_occurrences}")

count 1 '195' and its inverse '195' 
Total occurrences of 'TTAA' or its reverse complement: 390


In [42]:
# Define the query sequences
query = "ACTAAGT"
reverse_complement = str(Seq(query).reverse_complement())

In [43]:
# Find all matches for both sequences
def find_leftmost_offset(genome, pattern):
    try:
        return genome.index(pattern)  # 0-based offset of first occurrence
    except ValueError:
        return float('inf')  # Pattern not found

offset_query_flo = find_leftmost_offset(lambda_genome, query)
offset_rc_flo = find_leftmost_offset(lambda_genome, reverse_complement)

leftmost_offset = min(offset_query_flo, offset_rc_flo)
print(f"Leftmost offset of '{query}' or its reverse complement: {leftmost_offset}")

Leftmost offset of 'ACTAAGT' or its reverse complement: 26028


In [44]:
def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # Loop over all possible starting positions
        mismatches = 0
        for j in range(len(p)):           # Compare each character
            if t[i+j] != p[j]:
                mismatches += 1
                if mismatches > 2:
                    break
        if mismatches <= 2:
            occurrences.append(i)          # Record position if ≤2 mismatches
    return occurrences

# Load Lambda virus genome
lambda_genome = SeqIO.read('lambda_virus.fa', 'fasta').seq

# Search for 'TTCAAGCC' with up to 2 mismatches
matches = naive_2mm('TTCAAGCC', lambda_genome)
print(len(matches))  # Output the count of matches

191


In [45]:
matches = naive_2mm("AGGAGGTT", lambda_genome)
leftmost_offset = min(matches) if matches else None
print(f"Leftmost offset of 'AGGAGGTT' (≤2 mismatches): {leftmost_offset}")

Leftmost offset of 'AGGAGGTT' (≤2 mismatches): 49


In [46]:
upload = files.upload()
# ERR037900_1.first1000.fastq

Saving ERR037900_1.first1000.fastq to ERR037900_1.first1000.fastq


In [47]:
import numpy as np

# Initialize quality score statistics per position
qual_stats = {}

# Parse the FASTQ file
for record in SeqIO.parse("ERR037900_1.first1000.fastq", "fastq"):
    for pos, qual in enumerate(record.letter_annotations["phred_quality"]):
        if pos not in qual_stats:
            qual_stats[pos] = []
        qual_stats[pos].append(qual)

# Calculate mean quality per position and find the worst one
mean_quals = {pos: np.mean(quals) for pos, quals in qual_stats.items()}
worst_cycle = min(mean_quals.items(), key=lambda x: x[1])[0]

print(f"The problematic sequencing cycle is: {worst_cycle}")

The problematic sequencing cycle is: 66
